# ReRank

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
# !pip install cohere 
# !pip install weaviate-client

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
import cohere
co = cohere.Client(os.environ['COHERE_API_KEY'])

In [4]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(
    api_key=os.environ['WEAVIATE_API_KEY'])

In [5]:
client = weaviate.Client(
    url=os.environ['WEAVIATE_API_URL'],
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],
    }
)

## Dense Retrieval

In [6]:
from utils import dense_retrieval

In [7]:
query = "What is the capital of Canada?"

In [8]:
dense_retrieval_results = dense_retrieval(query, client)

UnexpectedStatusCodeException: Query was not successful! Unexpected status code: 500, with response body: None.

In [9]:
from utils import print_result

In [10]:
print_result(dense_retrieval_results)

NameError: name 'dense_retrieval_results' is not defined

## Improving Keyword Search with ReRank

In [11]:
from utils import keyword_search

In [12]:
query_1 = "What is the capital of Canada?"

In [13]:
query_1 = "What is the capital of Canada?"
results = keyword_search(query_1,
                         client,
                         properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results=3
                        )

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    print(result.get('text'))

i:0
Monarchy of Canada
In his 1990 book, "Continental Divide: the Values and Institutions of the United States and Canada," Seymour Martin Lipset argues that the presence of the monarchy in Canada helps distinguish Canadian identity from American identity. Since at least the 1930s, supporters of the Crown have held the opinion that the Canadian monarch is also one of the rare unified elements of Canadian society, focusing both "the historic consciousness of the nation" and various forms of patriotism and national love "[on] the point around which coheres the nation's sense of a continuing personality". Former Governor General Vincent Massey articulated in 1967 that the monarchy "is part of ourselves. It is linked in a very special way with our national life. It stands for qualities and institutions which mean Canada to every one of us and which for all our differences and all our variety have kept Canada Canadian." But, according to Arthur Bousfield and Gary Toffoli, Canadians were, th

In [14]:
query_1 = "What is the capital of Canada?"
results = keyword_search(query_1,
                         client,
                         properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results=500
                        )

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    #print(result.get('text'))

UnexpectedStatusCodeException: Query was not successful! Unexpected status code: 500, with response body: None.

In [15]:
def rerank_responses(query, responses, num_responses=10):
    reranked_responses = co.rerank(
        model = 'rerank-english-v2.0',
        query = query,
        documents = responses,
        top_n = num_responses,
        )
    return reranked_responses

In [16]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_1, texts)

In [17]:
for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()

i:0
RerankResult<document['text']: North America outside the zone of Spanish settlement was a contested area in the 17th century. Spain had founded small settlements in Florida and Georgia but nowhere near the size of those in New Spain or the Caribbean islands. France, The Netherlands, and Great Britain held several colonies in North America and the West Indies from the 17th century, 100 years after the Spanish and Portuguese established permanent colonies. The British colonies in North America were founded between 1607 (Virginia) and 1733 (Georgia). The Dutch explored the east coast of North America and began founding settlements in what they called New Netherland (now New York State.). France colonized what is now Eastern Canada, founding Quebec City in 1608. France's loss in the Seven Years' War resulted in the transfer of New France to Great Britain. The Thirteen Colonies, in lower British North America, rebelled against British rule in 1775, largely due to the taxation that Great

## Improving Dense Retrieval with ReRank

In [18]:
from utils import dense_retrieval

In [19]:
query_2 = "Who is the tallest person in history?"

In [20]:
results = dense_retrieval(query_2,client)

UnexpectedStatusCodeException: Query was not successful! Unexpected status code: 500, with response body: None.

In [21]:
for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    print(result.get('text'))
    print()

i:0
Monarchy of Canada
In his 1990 book, "Continental Divide: the Values and Institutions of the United States and Canada," Seymour Martin Lipset argues that the presence of the monarchy in Canada helps distinguish Canadian identity from American identity. Since at least the 1930s, supporters of the Crown have held the opinion that the Canadian monarch is also one of the rare unified elements of Canadian society, focusing both "the historic consciousness of the nation" and various forms of patriotism and national love "[on] the point around which coheres the nation's sense of a continuing personality". Former Governor General Vincent Massey articulated in 1967 that the monarchy "is part of ourselves. It is linked in a very special way with our national life. It stands for qualities and institutions which mean Canada to every one of us and which for all our differences and all our variety have kept Canada Canadian." But, according to Arthur Bousfield and Gary Toffoli, Canadians were, th

In [22]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_2, texts)

In [23]:
for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()

i:0
RerankResult<document['text']: North America outside the zone of Spanish settlement was a contested area in the 17th century. Spain had founded small settlements in Florida and Georgia but nowhere near the size of those in New Spain or the Caribbean islands. France, The Netherlands, and Great Britain held several colonies in North America and the West Indies from the 17th century, 100 years after the Spanish and Portuguese established permanent colonies. The British colonies in North America were founded between 1607 (Virginia) and 1733 (Georgia). The Dutch explored the east coast of North America and began founding settlements in what they called New Netherland (now New York State.). France colonized what is now Eastern Canada, founding Quebec City in 1608. France's loss in the Seven Years' War resulted in the transfer of New France to Great Britain. The Thirteen Colonies, in lower British North America, rebelled against British rule in 1775, largely due to the taxation that Great